Lo primero es poder cargar la información de los archivos brindados sin descomprimirlos

In [1]:
import gzip
import ast
import json
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

data = []
with gzip.open('steam_games.json.gz', 'rb') as f:
    for line in f:
        data.append(json.loads(line)) #json separa con doble comilla
games = pd.DataFrame(data)
f.close()

data = []
with gzip.open('user_reviews.json.gz', 'rb') as f:
    for line in f:
        data.append(ast.literal_eval(line.decode('utf-8'))) #json separa con simple comilla
reviews = pd.DataFrame(data)
f.close()

data = []
with gzip.open('users_items.json.gz', 'rb') as f:
    for line in f:
        data.append(ast.literal_eval(line.decode('utf-8'))) #json separa con simple comilla
items = pd.DataFrame(data)
f.close()

Análisis exploratorio inicial.

En una revición rápida pudimos ver que hay muchos vacios en el dataframe GAMES

In [2]:
print('filas vacias\n')
game_null=games.isnull().sum()
print(game_null)
print('\ntotal de filas: ', games.shape[0],'\n')
print('\n','*'*22,'\n')
print('Tenemos como conclución:')
print('Un mínimos de registros vacios de:',int(game_null.min()/games.shape[0]*100),'%')
print('Un Máximos de registros vacios de:',int(game_null.max()/games.shape[0]*100),'%')
print('\nPor ello procedemos a borrar los registro que están completamente vacios')

filas vacias

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

total de filas:  120445 


 ********************** 

Tenemos como conclución:
Un mínimos de registros vacios de: 73 %
Un Máximos de registros vacios de: 80 %

Por ello procedemos a borrar los registro que están completamente vacios


In [3]:
games2=games.dropna(how='all')

Analicemos un poco más este Dataframe. Veamos el nombre de sus columnas y que contienen.

In [4]:

column_name = games2.columns.tolist()
for i in range (len(column_name)):
    print(column_name[i])
games2.head()

publisher
genres
app_name
title
url
release_date
tags
reviews_url
specs
price
early_access
id
developer


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
88314,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN


In [5]:
print('cantidad de duplicados:',games2['id'].duplicated().sum())
id_duplicados = games2.index[games2['id'].duplicated()]
print(id_duplicados)


cantidad de duplicados: 2
Index([102883, 119271], dtype='int64')


la idea es tomar al dataframe games2 como tabla dimencional.
Por ello se dejan las columnas: genres, tags, specs anidadas para que no se repitan registros. Esto hay que tenerlo en cuenta a la hora de realizar las funciones.

__

Veamos los dos dataframes siguientes. "reviews" e "items"

In [6]:
column_name = reviews.columns.tolist()
for i in range (len(column_name)):
    print(column_name[i])


user_id
user_url
reviews


In [7]:
reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


La columna reviews se encuentra anidada.
Vamos a desanidarla.

In [8]:
reviews2 = reviews.explode('reviews')
reviews2.head(5)


,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."


La columna reviews es una columna anidada. Vamos a desanidarla.

In [9]:
reventado= pd.json_normalize(reviews2['reviews'])
reviews2.drop(columns='reviews', inplace= True)
reventado.reset_index(drop=True, inplace=True)  # Restablecer el índice
reviews2.reset_index(drop=True, inplace=True)   # Restablecer el índice
reviews2 = pd.concat([reviews2, reventado], axis=1)
reviews2.head(1)





,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...


Análisis de sentimiento ML

In [10]:
from textblob import TextBlob
def calcula_senti(text):
    if pd.isna(text):
        return 0.0, 0.0  
    else:
        text = str(text)
        blob = TextBlob(text)
        return blob.sentiment.polarity, blob.sentiment.subjectivity

polaridad = []
subjectividad = []

for review in reviews2['review']:
    polarity, subjectivity = calcula_senti(review)
    polaridad.append(polarity)
    subjectividad.append(subjectivity)

sentimientos = {'Polarity': polaridad, 'Subjectivity': subjectividad}
sentiments_df = pd.DataFrame(sentimientos)

st_ansis=[]
for i in range (len(sentiments_df['Polarity'])):
    if sentiments_df['Polarity'][i]<-0.33:
        st_ansis.append(0)
    elif sentiments_df['Polarity'][i]>0.33:
        st_ansis.append(2)
    else:
        st_ansis.append(1)
st_ansis=pd.DataFrame(st_ansis)
sentiments_df=pd.concat([sentiments_df, st_ansis],axis=1)

print(sentiments_df[0].value_counts())

0
1    45506
2    10140
0     3687
Name: count, dtype: int64


In [11]:
reviews2.drop(columns=['review'],inplace=True)
reviews2=pd.concat([reviews2,sentiments_df[0].rename('sentiment_analysis')],axis=1)
reviews2.head(20)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,1
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,1
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,1
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,1
5,js41637,http://steamcommunity.com/id/js41637,,"Posted November 29, 2013.",,239030,1 of 4 people (25%) found this review helpful,True,1
6,evcentric,http://steamcommunity.com/id/evcentric,,Posted February 3.,,248820,No ratings yet,True,1
7,evcentric,http://steamcommunity.com/id/evcentric,,"Posted December 4, 2015.","Last edited December 5, 2015.",370360,No ratings yet,True,2
8,evcentric,http://steamcommunity.com/id/evcentric,,"Posted November 3, 2014.",,237930,No ratings yet,True,2
9,evcentric,http://steamcommunity.com/id/evcentric,,"Posted October 15, 2014.",,263360,No ratings yet,True,0


Normalización del Dataframe Items

In [12]:
items2 = items.explode('items')

In [13]:
Itemzado= pd.json_normalize(items2['items'])
items2.drop(columns='items', inplace=True)
Itemzado.reset_index(drop=True, inplace=True)  # Restablecer el índice
items2.reset_index(drop=True, inplace=True)   # Restablecer el índice
items2 = pd.concat([items2, Itemzado], axis=1)
items2.head(1)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0


In [14]:
games2.iloc[:1].to_csv('1games.csv', index=False)
reviews2.iloc[:1].to_csv('1reviews.csv', index=False)
items2.iloc[:1].to_csv('1items.csv', index=False)

Generados los 3 archivos csv, se los une en un excel para poder estudiarlos y entender sus relaciones.

----------------

## Ahora empezamos a diseñar las funciones para los ENDPOINTS

----------------

ENDPOINT #1 Para este caso se busca optimizar una tabla.

consigna:

def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

----------------

Lo que se ve es que la columna de item2['item_id'] corresponde al mismo juego que games2['id']

In [15]:
items2[items2['item_id'] == '20'].head(5)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0,0.0
288,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,20,Team Fortress Classic,0.0,0.0
1303,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,20,Team Fortress Classic,0.0,0.0
1631,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,20,Team Fortress Classic,89.0,0.0
2549,NitemarePK,304,76561197990951820,http://steamcommunity.com/id/NitemarePK,20,Team Fortress Classic,0.0,0.0


In [16]:
games2[games2['id'] == '20']  

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120413,Valve,[Action],Team Fortress Classic,Team Fortress Classic,http://store.steampowered.com/app/20/Team_Fort...,1999-04-01,"[Action, FPS, Multiplayer, Classic, Shooter, C...",http://steamcommunity.com/app/20/reviews/?brow...,"[Multi-player, Valve Anti-Cheat enabled]",4.99,False,20,Valve


Con esa información podemos construir una tabla que contenga los valores enlistados de hora por juego y genero

In [17]:
horas= items2.groupby('item_id')['playtime_forever'].apply(list).reset_index() #Ponemos todas las horas juntas por año y géneros
horas = horas.rename(columns={'item_id':'id'})
games_horas= pd.merge(games2, horas, on='id', how='left')  #aplicamos un left Join
games_horas_pequeña=games_horas[['genres','release_date','playtime_forever']].copy()
games_horas_pequeña.isna().sum()

genres               3283
release_date         2067
playtime_forever    22943
dtype: int64

In [18]:
games_horas_pequeña = games_horas_pequeña[~games_horas_pequeña['playtime_forever'].isna()].reset_index() 
games_horas_pequeña.isna().sum()

index                 0
genres              534
release_date        406
playtime_forever      0
dtype: int64

In [19]:

año = games_horas_pequeña['release_date'].str.split('-').str[0]     #lo que nos interesa de release_date es solo el año
games_horas_pequeña.drop(columns='release_date', inplace=True)
games_horas_pequeña = pd.concat([games_horas_pequeña['genres'], año, games_horas_pequeña['playtime_forever']], axis=1) #con esto ya configuramso las columnas

# en Adelante es todo formateo del DataFrame
games_horas_pequeña = games_horas_pequeña[~(games_horas_pequeña['playtime_forever'].isna() | games_horas_pequeña['genres'].isna())].reset_index()
games_horas_pequeña = games_horas_pequeña.drop(columns=['index'])
games_horas_pequeña = games_horas_pequeña.rename(columns={'playtime_forever': 'horas'})
games_horas_pequeña['horas'] = games_horas_pequeña['horas'].apply(sum)
games_horas_pequeña.head(10)
games_horas_pequeña.to_csv('endpoint1.csv',index=False)             # CSV con tabla optimizada
games_horas_pequeña.head(3)

,genres,release_date,horas
0,"[Action, Indie, Racing]",1997,9319.0
1,[Action],1998,2682852.0
2,[Strategy],2006,27397.0


In [20]:
games_horas_pequeña.isnull().sum()

genres          0
release_date    5
horas           0
dtype: int64

Se generó un archivo horas.csv. 

Este contiene la info de consulta para la siguiente función, la que será usada en el EndPoint de FastAPI

In [21]:
'''def PlayTimeGenre( genero : str ):
    horas_por_año = {}
    for index, row in endpoint1.iterrows(): # endpoint1 es la variable que refiere al csv
        if genero in row['genres']:
            if row['release_date'] in horas_por_año:
                horas_por_año[row['release_date']] += row['horas']
            else:
                horas_por_año[row['release_date']] = row['horas']
    if horas_por_año:
        año_mas_jugado = max(horas_por_año, key=horas_por_año.get)
        horas_por_año = {}
        return f"año mas jugado para {genero} es: {año_mas_jugado}"
    else:
        return 'No se encontró dicho genero'

# Ejemplo de uso
string_busqueda = 'Action'
año_mas_jugado = obtener_año_mas_jugado(string_busqueda)
print(f"Año de lanzamiento con más horas jugadas para el género '{string_busqueda}': {año_mas_jugado}")'''

'def PlayTimeGenre( genero : str ):\n    horas_por_año = {}\n    for index, row in endpoint1.iterrows(): # endpoint1 es la variable que refiere al csv\n        if genero in row[\'genres\']:\n            if row[\'release_date\'] in horas_por_año:\n                horas_por_año[row[\'release_date\']] += row[\'horas\']\n            else:\n                horas_por_año[row[\'release_date\']] = row[\'horas\']\n    if horas_por_año:\n        año_mas_jugado = max(horas_por_año, key=horas_por_año.get)\n        return año_mas_jugado\n    else:\n        return None\n\n# Ejemplo de uso\nstring_busqueda = \'Action\'\naño_mas_jugado = obtener_año_mas_jugado(string_busqueda)\nprint(f"Año de lanzamiento con más horas jugadas para el género \'{string_busqueda}\': {año_mas_jugado}")'

---------------
ENDPOINT #2

Creamos los dataframe a partir de la base de datos de Steam

consigna:

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

In [22]:
#Tabla 1, tomamos el playtime_forever
items_end2= items2[['user_id','item_id','playtime_forever']]
items_end2.head()
#Tabla 2, tomamos el año
reviews_end2= reviews2[['user_id','item_id','posted']]
años= reviews_end2['posted'].str.split(',', expand=True)[1]
años= año.str.replace('.', '')
reviews_end2=pd.concat([reviews_end2,años], axis=1)
reviews_end2.drop(columns={'posted'}, inplace= True)
#Tabla 3, tomamos los generos
genres_end2= games2[['genres','id']]


In [23]:
#unión de tablas
end2data= pd.merge(items_end2, reviews_end2, on=['user_id','item_id'], how='inner')
end2data=end2data.groupby(['user_id','item_id','release_date']).sum().reset_index()
end2data.head()
end2data=pd.merge(end2data, genres_end2, left_on= 'item_id', right_on= 'id', how= 'inner')
end2data = end2data.dropna(subset=['genres'])
end2data.drop(columns=['item_id','id'],inplace=True)


In [24]:
end2data = end2data.explode('genres')
end2data.to_csv('endpoint2.csv', index=False)
end2data.head()


,user_id,release_date,playtime_forever,genres
0,-Azsael-,2014,11889.0,Indie
0,-Azsael-,2014,11889.0,Strategy
1,stverbal,2015,1231.0,Indie
1,stverbal,2015,1231.0,Strategy
154,00690069006900,1994,22511.0,Action


In [25]:
def UserForGenre(genero: str):
    playtime_por_usuario = {}
    playtime_por_año = {}

    for index, registro in endpoint2.iterrows():
        if registro['genres'] == genero:
            user_id = registro['user_id']
            playtime = registro['playtime_forever']
            if user_id in playtime_por_usuario:
                playtime_por_usuario[user_id] += playtime
            else:
                playtime_por_usuario[user_id] = playtime

            año = registro['release_date'][:4]  # Obtenemos solo el año
            if user_id in playtime_por_año:
                playtime_por_año[user_id].setdefault(año, 0)
                playtime_por_año[user_id][año] += playtime
            else:
                playtime_por_año[user_id] = {año: playtime}
        else: return print("No se encuentra el genero")        

    max_playtime = max(playtime_por_usuario.values())
    usuario_max_playtime = [user_id for user_id, playtime in playtime_por_usuario.items() if round(playtime, 2) == max_playtime]

    años_playtime_max = playtime_por_año.get(usuario_max_playtime[0], {})
    # Convertir el playtime de cada año a horas
    for año, playtime in años_playtime_max.items():
        años_playtime_max[año] = playtime / 60  # Suponiendo que las unidades de juego son horas
    return f"Usuario que mas jugó: {usuario_max_playtime[0] if usuario_max_playtime else None}. Los años y horas {años_playtime_max}"        

---------------------------------------------
ENDPOINT #3

consigna:

def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

Agregar la columna del año donde fue postiado la reseña [año_posted]

In [26]:
año_posted = reviews2['posted'].str.extract(r'(\d{4})')
reviews_con_posted=pd.concat([reviews2,año_posted],axis=1)
reviews_con_posted.rename(columns={0:'año_posted'}, inplace=True)
reviews_con_posted.head(3)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analysis,año_posted
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,1,2011
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,2,2011
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,1,2011


Contabilizar la cantidad de veces recomendado

In [27]:
rank_data = reviews_con_posted[['item_id','año_posted','recommend']]
rank_data = pd.DataFrame(rank_data.groupby(['año_posted','item_id']).size())
rank_data.reset_index(inplace=True)
rank_data.rename(columns={0:'count'},inplace=True)
rank_data.head(10)

,año_posted,item_id,count
0,2010,10150,1
1,2010,10500,1
2,2010,11170,1
3,2010,1250,6
4,2010,12900,1
5,2010,13570,1
6,2010,15320,1
7,2010,17410,1
8,2010,17450,1
9,2010,21970,1


relación entre nombre de juego y su identificación [id]

In [28]:
juego_id = games2[['app_name','id']]
juego_id.head()

,app_name,id
88310,Lost Summoner Kitty,761140
88311,Ironbound,643980
88312,Real Pool 3D - Poolians,670290
88313,弹炸人2222,767400
88314,Log Challenge,773570


Reemplazar la id por el nombre del juego

In [29]:
rank_data = pd.merge(rank_data, juego_id, left_on='item_id', right_on='id', how='left')
rank_data.drop(['item_id', 'id'], axis=1, inplace=True)
rank_data.to_csv('endpoint3.csv',index=False)
rank_data.head(10)

,año_posted,count,app_name
0,2010,1,Prototype™
1,2010,1,Empire: Total War™
2,2010,1,NaN
3,2010,6,Killing Floor
4,2010,1,AudioSurf
5,2010,1,Tom Clancy's Splinter Cell Chaos Theory®
6,2010,1,IL-2 Sturmovik: 1946
7,2010,1,Mirror's Edge™
8,2010,1,Dragon Age: Origins
9,2010,1,NaN


Función para ENDPOINT 3

consigna: def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

In [30]:
'''def UsersRecommend (año:int):
    data_valor = endpoint3[endpoint3['año_posted'] == año] # endpoint3 es la variable que refiere al csv
    data_valor = data_valor.sort_values(by='count', ascending=False) # Ordenar por columna count
    max_game = data_valor.iloc[0]['app_name'] # el 1°
    second_max_game = data_valor.iloc[1]['app_name'] # el 2°
    third_max_game = data_valor.iloc[2]['app_name'] # el 3°
    return print("Primero más recomendado:", max_game, "Segundo más recomendado:", second_max_game, "Tercer más recomendado:", third_max_game)
'''

'def UsersRecommend (año:int):\n    data_valor = endpoint3[endpoint3[\'año_posted\'] == año] # endpoint3 es la variable que refiere al csv\n    data_valor = data_valor.sort_values(by=\'count\', ascending=False) # Ordenar por columna count\n    max_game = data_valor.iloc[0][\'app_name\'] # el 1°\n    second_max_game = data_valor.iloc[1][\'app_name\'] # el 2°\n    third_max_game = data_valor.iloc[2][\'app_name\'] # el 3°\n    return print("Primero más recomendado:", max_game, "Segundo más recomendado:", second_max_game, "Tercer más recomendado:", third_max_game)\n'

-----------------------------
Función para ENDPOINT 4

consigna:def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

In [31]:
'''def UsersNotRecommend (ano:int):
    if ano in endpoint3['año_posted']:
        data_valor = endpoint3[endpoint3['año_posted'] == ano] # Filtrar valor de la función
        data_valor = data_valor.sort_values(by='count', ascending=True) # Ordenar por columna count
        min_game = data_valor.iloc[0]['app_name'] # el 1°
        second_min_game = data_valor.iloc[1]['app_name'] # el 2°
        third_min_game = data_valor.iloc[2]['app_name'] # el 3°
        
        return "Primero más recomendado:", min_game, "Segundo más recomendado:", second_min_game, "Tercer más recomendado:", third_min_game
    else: return 'no se encuentra el año'
    '''

'def UsersNotRecommend (año:int):\n    data_valor = endpoint3[endpoint3[\'año_posted\'] == año] # Filtrar valor de la función\n    data_valor = data_valor.sort_values(by=\'count\', ascending=True) # Ordenar por columna count\n    max_game = data_valor.iloc[0][\'app_name\'] # el 1°\n    second_max_game = data_valor.iloc[1][\'app_name\'] # el 2°\n    third_max_game = data_valor.iloc[2][\'app_name\'] # el 3°\n    return print("Primero más recomendado:", max_game, "Segundo más recomendado:", second_max_game, "Tercer más recomendado:", third_max_game)'

---------------------------------------------------------
ENDPOINT 5

consigna: def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}



In [32]:
#Tabla con ID de juego y año de lanzamiento
games_end5=games2[['release_date','id']].copy()
games_end5_año=games_end5['release_date'].str.split('-', expand=True)[0]
games_end5.drop(columns='release_date', inplace=True)
games_end5=pd.concat([games_end5, games_end5_año], axis=1)
games_end5.rename(columns={games_end5.columns[1]:'año'}, inplace=True)
games_end5.head(2)

,id,año
88310,761140,2018
88311,643980,2018


In [33]:
#Valores de análisis de sentimiento
reviews_end5=reviews2[['item_id','sentiment_analysis']]
reviews_end5.groupby('item_id')['sentiment_analysis'].count()
conteo = reviews_end5.pivot_table(index='item_id', columns='sentiment_analysis', aggfunc='size', fill_value=0)
conteo.columns = ['sentiment_0', 'sentiment_1', 'sentiment_2']
conteo.reset_index(inplace=True)
sentiment_end5 = pd.merge(games_end5, conteo, left_on='id', right_on='item_id', how='inner')
sentiment_end5.drop(columns={'item_id','id'}, inplace=True)
sentiment_end5 = sentiment_end5.groupby('año').sum().reset_index()
sentiment_end5.iloc[35]

año            Mar 2010
sentiment_0           0
sentiment_1           1
sentiment_2           0
Name: 35, dtype: object

In [34]:
#en la columna año se puede ver que hay registros donde aparece el mes y luego el año
#vamos a eliminar el mes de año y reagrupar sus registros
import re

def eliminar_mes(fecha):
    return re.sub(r'\b\w+\b\s', '', fecha)

sentiment_end5['año'] = sentiment_end5['año'].apply(eliminar_mes)
sentiment_end5 = sentiment_end5.groupby('año').sum().reset_index()
#sentiment_end5['año'].duplicated().count()
sentiment_end5.to_csv('endpoint5.csv', index=False)
sentiment_end5.head(10)


,año,sentiment_0,sentiment_1,sentiment_2
0,1989,0,1,0
1,1990,0,3,2
2,1991,0,1,0
3,1992,0,3,0
4,1993,0,4,1
5,1994,0,14,1
6,1995,0,15,1
7,1996,2,5,1
8,1997,2,40,6
9,1998,3,76,17


Función para ENDPOINT 5

In [35]:
'''def sentiment_analysis(ano: int):
    list= endpoint5['año'].tolist()
    if ano in list:  # endpoint5 variable que refiere al csv
        return f"Negativos: {endpoint5.iloc[list.index(ano)]['sentiment_0']}, Neutros: {endpoint5.iloc[list.index(ano)]['sentiment_1']}, Positivos: {endpoint5.iloc[list.index(ano)]['sentiment_2']}"
    else:
        return 'No se encontró valor para:', ano'''

------------------------------------------------------------------
RECOMENDACIONES POR JUEGO:

Para no agrandar mucho el dataset, usaremos las columnas:

GAME2[GENRES] llevado a dummies.

GAME2[PRICE] normalizado


ITEMS[PLAYTIME_FOREVER] sumado para cada juego

In [36]:

# obtenemos las subtablas
rec_genres=games2[['id','genres','price']]
rec_play=items2[['item_id','playtime_forever']]

In [37]:

rec_gen_dum = pd.get_dummies(rec_genres['genres'].apply(pd.Series).stack()).groupby(level=0).sum()
rec_gen_dum.fillna(0, inplace=True)
rec_genres = pd.concat([rec_genres,rec_gen_dum], axis=1)
rec_genres.head(2)
rec_play = rec_play.groupby('item_id').sum()

In [38]:
rec_play=pd.merge(rec_genres, rec_play, left_on= 'id', right_on= 'item_id', how= 'inner')
rec_play.shape

(9192, 26)

In [39]:
rec_play.drop(columns='genres',inplace=True)

Ahora hay que normalizar la columna 'price'

In [40]:
#hay valores dentro de la columna que no son flotantes. ellos son:
rec_play['price'].replace('Starting at $449.00','449.00')
rec_play['price'] = rec_play['price'].apply(lambda x: 0.0 if isinstance(x, str) else x)

import math
rec_play['price'] = rec_play['price'].apply(lambda x: 0.0 if isinstance(x, float) and math.isnan(x) else x) # esta linea es para reemplazar los nan, mi computador tiene problemas con la librería de numpy X(
rec_play['price'].unique()

array([9.9900e+00, 6.9900e+00, 0.0000e+00, 3.9990e+01, 4.9900e+00,
       1.9990e+01, 7.4900e+00, 1.4990e+01, 8.9900e+00, 2.9900e+00,
       1.2990e+01, 5.9900e+00, 7.9900e+00, 2.4900e+00, 1.5990e+01,
       3.9900e+00, 1.9900e+00, 9.9000e-01, 6.1990e+01, 2.9990e+01,
       2.4990e+01, 1.1990e+01, 1.3990e+01, 7.4760e+01, 3.2990e+01,
       9.9990e+01, 1.4950e+01, 5.9990e+01, 1.3980e+01, 3.4990e+01,
       2.9960e+01, 7.9990e+01, 1.6990e+01, 7.7171e+02, 1.8990e+01,
       4.9990e+01, 1.0990e+01, 9.8000e-01, 1.3992e+02, 1.7990e+01,
       8.9000e-01, 5.0000e-01, 2.1990e+01, 2.9999e+02, 1.2900e+00,
       4.4990e+01, 3.0000e+00, 1.5000e+01, 4.9000e+01, 1.0930e+01,
       2.0000e+00, 4.0000e+00, 9.0000e+00, 1.1999e+02, 1.5000e+00,
       6.9990e+01, 2.6990e+01, 1.6060e+01, 1.8900e+01, 2.8900e+00,
       8.7940e+01, 2.0990e+01, 8.9800e+00, 9.6900e+00, 4.9000e-01,
       9.9800e+00, 9.9500e+00, 2.0000e+01, 3.1990e+01, 1.0000e+00,
       1.2890e+01, 1.8700e+00, 1.0000e+01, 7.4990e+01, 2.3960e

In [41]:
min_price = rec_play['price'].min()
max_price = rec_play['price'].max()
rec_play['price'] = (rec_play['price'] - min_price) / (max_price - min_price)
print('max',rec_play['price'].max())
print('min',rec_play['price'].min())

max 1.0
min 0.0


Ahora hay que normalizar la columna 'playtime_forever'

In [42]:
#Por suerte todos los datos son flotantes y no hay nulos
rec_play['playtime_forever'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 9192 entries, 0 to 9191
Series name: playtime_forever
Non-Null Count  Dtype  
--------------  -----  
9192 non-null   float64
dtypes: float64(1)
memory usage: 71.9 KB


In [43]:
#ptf = playtime_forever
min_ptf = rec_play['playtime_forever'].min()
max_ptf = rec_play['playtime_forever'].max()
rec_play['playtime_forever'] = (rec_play['playtime_forever'] - min_ptf) / (max_ptf - min_ptf)

print(rec_play['playtime_forever'].max())
print(rec_play['playtime_forever'].min())


1.0
0.0


In [80]:
rec_play.reset_index()
rec_play.fillna(0, inplace=True)
rec_play.head(10)

,id,price,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,...,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,playtime_forever
0,282010,0.012945,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000012
1,70,0.012945,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003417
2,1640,0.009058,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000035
3,1630,0.009058,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000027
4,2400,0.012945,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001335
5,374970,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
6,3800,0.012945,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000017
7,1520,0.012945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000282
8,2420,0.012945,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000247
9,4000,0.012945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.571034


In [81]:
#genero el csv
rec_play.to_csv('recomendado.csv')

Función de Recomendación

In [47]:
recomendados = pd.read_csv('C:\\ds\\henry\\PI1\\PI MLOps - STEAM\\recomendado.csv')


In [68]:
recomendados.iloc[:,2:26]

,price,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,...,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,playtime_forever
0,0.012945,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.186855e-05
1,0.012945,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.416844e-03
2,0.009058,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.489245e-05
3,0.009058,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.688668e-05
4,0.012945,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.334917e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9187,0.009058,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.725006e-07
9188,0.006466,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.666406e-04
9189,0.025904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.321279e-06
9190,0.019424,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.446332e-04


In [49]:
'''def recomendacion_juego( id : int):
    data = pd.read_csv('recomendado.csv')
    df = pd.DataFrame(data)

    if id not in df['id'].values:
        return print("No se encuentra ese ID")    

    id_consultado = df[df['id'] == id].iloc[:,2:26].values.reshape(1, -1)
    otros_id = df[df['id'] != 1].iloc[:,2:26].values

    similar = cosine_similarity(id_consultado, otros_id)

    mas_similares = similar.argsort()[0][-5:][::-1]
    ids_mas_similares = df.loc[df.index.isin(mas_similares), 'id'].tolist()
    return f"Los 5 IDs más similares: {ids_mas_similares}"
'''